<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Climate-Data" data-toc-modified-id="Climate-Data-1">Climate Data</a></span></li><li><span><a href="#MODIS" data-toc-modified-id="MODIS-2">MODIS</a></span></li><li><span><a href="#Sentinel-2" data-toc-modified-id="Sentinel-2-3">Sentinel-2</a></span></li></ul></div>

In [ ]:
!pip install geopy
!pip install geehydro
!pip install folium
!pip install -U -q git+https://github.com/mccarthyryanc/folium_gee.git
!pip install earthengine-api

In [5]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-pastel')
%matplotlib inline

import ee 
import folium
import foliumgee
import geehydro
import geopy
# import geemap
import geemap.eefolium as geemap

# import proplot as plot 
from ipygee import*
from pandas.plotting import register_matplotlib_converters

import requests
import os
import warnings

from IPython.display import HTML
from IPython.display import display

from datetime import datetime as dt
from IPython.display import Image

In [6]:
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AY0e-g4huwwtZPMAKnoAwKG36jJkkLyTNwSNI0CNwHwlz9dFwubnAmQ0T4o

Successfully saved authorization token.


In [18]:
# Your Earth Engine username.  This is used to import a classified image
# into your Earth Engine assets folder.
USER_NAME = 'vee_vargas'

# Cloud Storage bucket into which training, testing and prediction 
# datasets will be written.  You must be able to write into this bucket.
OUTPUT_BUCKET = 'aus_fire_bucket'

from google.cloud import storage

def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    # bucket_name = "your-bucket-name"
    # source_file_name = "local/path/to/file"
    # destination_blob_name = "storage-object-name"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)
#     blob.upload_from_file() 
#     blob.upload_from_string() 

    print(
        "File {} uploaded to {}.".format(
            source_file_name, destination_blob_name
        )
    )


In [8]:
import sys
from google.cloud import storage


def view_bucket_iam_members(bucket_name):
    """View IAM Policy for a bucket"""
    # bucket_name = "your-bucket-name"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)

    policy = bucket.get_iam_policy(requested_policy_version=3)

    for binding in policy.bindings:
        print("Role: {}, Members: {}".format(binding["role"], binding["members"]))
        
# view_bucket_iam_members(bucket_name=sys.argv[1])

# def list_file_archived_generations(bucket_name):
#     """Lists all the blobs in the bucket with generation."""
#     # bucket_name = "your-bucket-name"

#     storage_client = storage.Client()

#     blobs = storage_client.list_blobs(bucket_name)

#     for blob in blobs:
#         print("{},{}".format(blob.name, blob.generation))

# list_file_archived_generations(bucket_name)

In [9]:
ee.FeatureCollection('LT05_L1TP_160043_20011208_20180930_01_T1')

In [23]:
city = "Melbourne"
country = 'Australia'

# get location (Nominatim geocoder for OpenStreetMap data)
locator = geopy.geocoders.Nominatim(user_agent="My app") 
country = locator.geocode(country)
location = [country.latitude, country.longitude]
print(country, "\n[lat, long]:", location)
# city = locator.geocode(city)
# location = [city.latitude, city.longitude]
# print(city, "\n[lat, long]:", location)

Australia 
[lat, long]: [-24.7761086, 134.755]


In [28]:
aus_map = folium.Map(location=location)
# aus_map

In [ ]:
# Cloud masking function.
def maskL8sr(image):
    cloudShadowBitMask = ee.Number(2).pow(3).int()
    cloudsBitMask = ee.Number(2).pow(5).int()
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
        qa.bitwiseAnd(cloudsBitMask).eq(0))
    return image.updateMask(mask).select(BANDS).divide(10000)

# The image input data is a 2018 cloud-masked median composite.
image = L8SR.filterDate('2018-01-01', '2018-12-31').map(maskL8sr).median()

# Use folium to visualize the imagery.
mapid = image.getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3})
aus_map = folium.Map(location=location)

folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='median composite',
  ).add_to(aus_map)
aus_map.add_child(folium.LayerControl())
aus_map

# Climate Data

- Australian Bureau of Meterology (BoM): climate data, temperature and rainfall (weather stations near Sydney, Perth, Brisbane, Canberra, and Adelaide)

> rainfall 	- 	rainfall in millimeters

> period 	- 	how many days was it collected across

> quality 	- 	certified quality or not

Copernicus Climate Change (https://cds.climate.copernicus.eu/#!/home) 

Atmosphere Monitoring Service (https://atmosphere.copernicus.eu/data) 

In [20]:
url = "http://www.bom.gov.au/web01/ncc/www/cli_chg/timeseries/rain/0112/aus/latest.txt"
rainfall = pd.read_csv(url)
rainfall.head()

,station_code,city_name,year,month,day,rainfall,period,quality,lat,long,station_name
0,9151,Perth,1967,1,1,NaN,NaN,NaN,-31.96,115.79,Subiaco Wastewater Treatment Plant
1,9151,Perth,1967,1,2,NaN,NaN,NaN,-31.96,115.79,Subiaco Wastewater Treatment Plant
2,9151,Perth,1967,1,3,NaN,NaN,NaN,-31.96,115.79,Subiaco Wastewater Treatment Plant
3,9151,Perth,1967,1,4,NaN,NaN,NaN,-31.96,115.79,Subiaco Wastewater Treatment Plant
4,9151,Perth,1967,1,5,NaN,NaN,NaN,-31.96,115.79,Subiaco Wastewater Treatment Plant


In [ ]:
# Annual rainfall (mm)
url = "http://www.bom.gov.au/web01/ncc/www/cli_chg/timeseries/rain/0112/aus/latest.txt"
response_string = requests.get(url).text.split("\n")[:-1]
col1 = [int(item[0:4]) for item in response_string]
col2 = [float(item[14:]) for item in response_string]

In [ ]:
dict_of_cols = {"annual_rainfall_year": col1, "annual_rainfall_mm": col2}
df_annual_rainfall = pd.DataFrame(data=dict_of_cols)
df_annual_rainfall.to_csv("data/df_annual_rainfall.csv")
df_annual_rainfall.tail(2)

In [21]:
temperature = pd.read_csv('http://www.bom.gov.au/climate/data/index.shtml??zoom=1&lat=-26.9635&lon=133.4635&layers=B0000TFFFFFFFFFTFFFFFFFFFFFFFFFFTTT&dp=IDC10001&p_nccObsCode=201&p_display_type=dailyDataFile')
temperature.head()

,city_name,date,temperature,temp_type,site_name
0,PERTH,1910-01-01,26.7,max,PERTH AIRPORT
1,PERTH,1910-01-02,27.0,max,PERTH AIRPORT
2,PERTH,1910-01-03,27.5,max,PERTH AIRPORT
3,PERTH,1910-01-04,24.0,max,PERTH AIRPORT
4,PERTH,1910-01-05,24.8,max,PERTH AIRPORT


In [36]:
path_weather = "data/HistoricalWeather.csv"
weather_df = pd.read_csv(path_weather)

weather_df = weather_df.rename(columns={"count()[unit: km^2]": "Area", "min()": "Min",
                                       "max()": "Max", "mean()": "Mean", "variance()": "Variance"})
weather_df['Date'] = pd.to_datetime(weather_df['Date'])
weather_df.Parameter.unique()

array(['Precipitation', 'RelativeHumidity', 'SoilWaterContent',
       'SolarRadiation', 'Temperature', 'WindSpeed'], dtype=object)

# MODIS 

In [5]:
import pandas as pd

In [21]:
# modis_c6.describe()

In [12]:
modis_fire = pd.read_csv('../data/fire_archive_M6_174304.csv')
modis_fire

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type
0,-15.8931,136.6094,324.4,1.2,1.1,2015-01-01,104,Terra,MODIS,28,6.03,302.3,11.6,D,0
1,-15.6032,136.6719,315.4,1.2,1.1,2015-01-01,104,Terra,MODIS,64,6.03,297.1,10.1,D,3
2,-14.9764,145.2801,320.6,2.0,1.4,2015-01-01,104,Terra,MODIS,24,6.03,294.8,25.0,D,0
3,-18.5115,139.5995,331.8,1.0,1.0,2015-01-01,105,Terra,MODIS,37,6.03,305.0,19.1,D,0
4,-18.0797,122.6967,313.7,3.2,1.7,2015-01-01,243,Terra,MODIS,31,6.03,292.9,39.2,D,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240187,-20.7341,139.4823,303.8,1.2,1.1,2016-01-01,1552,Aqua,MODIS,57,6.03,290.8,7.5,N,2
240188,-24.0261,147.7287,301.4,1.9,1.3,2016-01-01,1553,Aqua,MODIS,7,6.03,287.1,16.5,N,0
240189,-22.9036,117.7854,310.8,1.0,1.0,2016-01-01,1732,Aqua,MODIS,73,6.03,298.1,7.0,N,0
240190,-24.9992,152.4644,330.9,1.9,1.3,2016-01-01,2341,Terra,MODIS,81,6.03,296.1,61.6,D,0


In [24]:
modis_fire.describe()

,latitude,longitude,brightness,scan,track,acq_time,confidence,version,bright_t31,frp,type
count,240192.000000,240192.000000,240192.000000,240192.000000,240192.000000,240192.000000,240192.000000,2.401920e+05,240192.000000,240192.000000,240192.000000
mean,-19.517656,132.938033,332.061722,1.684701,1.234639,566.140167,70.089174,6.030000e+00,303.868348,64.200927,0.020017
std,6.812452,9.026544,20.909181,0.860465,0.259477,489.164878,22.482179,2.268768e-11,9.771162,138.703286,0.213727
min,-43.360100,113.129000,300.000000,1.000000,1.000000,0.000000,0.000000,6.030000e+00,266.900000,0.000000,0.000000
25%,-22.166650,127.116250,318.700000,1.100000,1.000000,206.000000,55.000000,6.030000e+00,297.100000,14.500000,0.000000
50%,-17.342800,132.237650,328.600000,1.300000,1.100000,442.000000,73.000000,6.030000e+00,304.200000,28.200000,0.000000
75%,-14.500575,137.303600,340.500000,2.000000,1.400000,546.000000,88.000000,6.030000e+00,309.900000,60.500000,0.000000
max,-9.135800,153.542500,506.900000,4.800000,2.000000,2359.000000,100.000000,6.030000e+00,400.100000,5745.200000,3.000000


In [ ]:
# Convert dtype of acq_date to datetime and create new column New Date
df['New Date'] = pd.to_datetime(df['acq_date'], infer_datetime_format = True)

# get Year 
df['year'] = pd.DatetimeIndex(df['New Date']).year

# Important information
df = df[['latitude','longitude','year']]

# Round latitude, longitude to .1 
decimals = pd.Series([1, 1], index=['latitude', 'longitude'])
df.round(decimals).sort_values('year')

#save
# df.round(decimals).to_csv('australia_rounded.csv', index = False)

In [ ]:
def scale_factor(image):
# scale factor for the MODIS MOD13Q1 product
return image.multiply(0.0001).copyProperties(image, 
                                             ['system:time_start'])# mapping function to multiply by the scale factor
scaled_evi = evi.map(scale_factor)
scaled_ndvi = ndvi.map(scale_factor)

# Sentinel-2

In [38]:
s2.getInfo()

{'type': 'Image', 'bands': []}

In [36]:
type(s2)

ee.image.Image

In [27]:
import arcgis
from arcgis.gis import GIS
from arcgis.raster.functions import *
from arcgis.geoanalytics.use_proximity import create_buffers
from arcgis.geoenrichment import enrich
from arcgis.features import SpatialDataFrame
from arcgis.raster.analytics import create_image_collection
from arcgis.raster.analytics import list_datastore_content

In [28]:
gis = GIS(username="Vanessa_Vargas_LearnArcGIS")

Enter password: ········


In [29]:
my_map = gis.map("australia")
my_map.basemap = "dark-gray-vector"
my_map

MapView(layout=Layout(height='400px', width='100%'))

In [30]:
fire_result = gis.content.search("MODIS fire type:Feature Layer", outside_org=True)
fire_result

[<Item title:"Satellite (VIIRS) Thermal Hotspots and Fire Activity" type:Feature Layer Collection owner:esri_livefeeds2>,
 <Item title:"Satellite (MODIS) Thermal Hotspots and Fire Activity" type:Feature Layer Collection owner:esri_livefeeds2>,
 <Item title:"MODIS Thermal (Last 48 hours)" type:Feature Layer Collection owner:Dlpratt>,
 <Item title:"VIIRS_VNP14IMGTDL_NRT_Europe_24h_200918" type:Feature Layer Collection owner:support@esri>,
 <Item title:"Join_Features_to_Satellite_(MODIS)_Thermal_Hotspots_and_Fire_Activity" type:Feature Layer Collection owner:iZhuravleva>,
 <Item title:"Find_Clusters_for_Satellite_(MODIS)_Thermal_Hotspots_and_Fire_Activity" type:Feature Layer Collection owner:Ixchal_Gonzalez_LearnStudent>,
 <Item title:"Describe_distribution_of_Find_Clusters_for_Satellite_(MODIS)_Thermal_Hotspots_and_Fire_Activity" type:Feature Layer Collection owner:Ixchal_Gonzalez_LearnStudent>,
 <Item title:"Buffer_of_Fire_in_Australia_Jan_8_MODIS_C6_Australia_and_New_Zealand_24h" type:

In [31]:
precip_result = gis.content.search("gldas", outside_org=True)
precip_result

[<Item title:"Africa Precipitation (2016-2018)" type:Web Map owner:esri_africa>,
 <Item title:"Scientific Data Workflows" type:Code Sample owner:RasterDevTeam>,
 <Item title:"Monthly Precipitation" type:Web Map owner:esri_observations>,
 <Item title:"Monthly Soil Moisture" type:Web Map owner:esri_observations>,
 <Item title:"Water Balance App" type:Web Mapping Application owner:esri_hydro>,
 <Item title:"Monthly Snow Pack" type:Web Map owner:esri_observations>,
 <Item title:"Monthly Runoff" type:Web Map owner:esri_observations>,
 <Item title:"Monthly Evapotranspiration" type:Web Map owner:esri_observations>,
 <Item title:"GLDAS Evapotranspiration 2000 - Present" type:Imagery Layer owner:esri>,
 <Item title:"Daily Evapotranspiration" type:Web Mapping Application owner:curate88>]

In [32]:
fire_result[1].id

'b8f4033069f141729ffb298b7418b653'

In [34]:
fire_layer = gis.content.get(fire_result[1].id)
my_map.add_layer(fire_layer)
my_map

MapView(jupyter_target='notebook', layout=Layout(height='400px', width='100%'), ready=True)

In [ ]:
# create color column to correspond to type
colors = ["red","yellow","orange", "green"]
indices = sorted(list(df["type"].unique()))
df["color"] = df["type"].apply(lambda x: 
               colors[indices.index(x)])

## scaling the size
scaler = preprocessing.MinMaxScaler(feature_range=(3,15))
df["size"] = scaler.fit_transform(
               df['brightness'].values.reshape(-1,1)).reshape(-1)

In [ ]:
landsat = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")

# setting the Area of Interest (AOI)
Ituna_AOI = ee.Geometry.Rectangle([-51.84448, -3.92180,
                                   -52.23999, -4.38201])
# filter area
landsat_AOI = landsat.filterBounds(Ituna_AOI)

landsat = landsat.filterDate('2019-07-01','2019-12-01')

In [35]:
# create a Spatially Enabled DataFrame object
sdf = pd.DataFrame.spatial.from_layer(fire_layer.layers[0])
sdf.head()

,OBJECTID,BRIGHTNESS,SCAN,TRACK,SATELLITE,CONFIDENCE,VERSION,BRIGHT_T31,FRP,ACQ_DATE,DAYNIGHT,SHAPE
0,1,315.6,1.6,1.2,T,0,6.0NRT,287.3,12.5,2021-01-10 00:55:00,D,"{""x"": 15216149.877099998, ""y"": -1414639.968199..."
1,2,342.6,2.2,1.4,T,90,6.0NRT,298.9,107.7,2021-01-10 00:55:00,D,"{""x"": 14932953.0924, ""y"": -1752090.5659000017,..."
2,3,344.7,2.2,1.4,T,91,6.0NRT,299.6,119.8,2021-01-10 00:55:00,D,"{""x"": 14935179.482299998, ""y"": -1752552.754900..."
3,4,331.5,1.0,1.0,T,67,6.0NRT,308.6,13.4,2021-01-10 00:55:00,D,"{""x"": 15526731.256399997, ""y"": -2360427.103700..."
4,5,312.1,1.1,1.1,T,77,6.0NRT,292.6,10.5,2021-01-10 01:35:00,N,"{""x"": -4533597.582, ""y"": -2626456.662799999, ""..."


In [19]:
import pandas as pd
import os
import numpy as np

files = ['df_2.csv','df_3.csv']

df = pd.read_csv('data/df_2.csv')

df['confidence'] = pd.to_numeric(df['confidence'])

for index, row in df.iterrows():
    if df.loc[index,'confidence'] <= 33.33:
        df.loc[index,'confidence'] = '0_low'
    elif (df.loc[index,'confidence'] > 33.33) & (df.loc[index,'confidence'] <= 66.66):
        df.loc[index,'confidence'] = '1_med'
    elif df.loc[index,'confidence'] > 66.66:
        df.loc[index,'confidence'] = '2_high'

df.reset_index(drop = True)
df.to_csv('data/df_conf.csv')
print('saved')

saved


In [18]:
df = pd.read_csv('data/df_2.csv')
df

,Unnamed: 0,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp
0,0,-11.8070,142.0583,313.0,1.0,1.0,2019-08-01,56,Terra,MODIS,48,6.3,297.3,6.6
1,1,-11.7924,142.0850,319.3,1.0,1.0,2019-08-01,56,Terra,MODIS,71,6.3,297.3,11.3
2,2,-12.8398,132.8744,311.6,3.1,1.7,2019-08-01,57,Terra,MODIS,42,6.3,298.7,23.1
3,3,-14.4306,143.3035,310.1,1.1,1.1,2019-08-01,57,Terra,MODIS,33,6.3,296.1,6.5
4,4,-12.4953,131.4897,310.3,4.0,1.9,2019-08-01,57,Terra,MODIS,36,6.3,298.8,27.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36006,36006,-28.0656,131.6334,312.7,1.2,1.1,2019-09-30,1644,Aqua,MODIS,85,6.3,292.9,14.2
36007,36007,-28.2625,128.5494,304.9,1.0,1.0,2019-09-30,1644,Aqua,MODIS,27,6.3,290.9,6.4
36008,36008,-28.2941,128.5123,301.7,1.0,1.0,2019-09-30,1644,Aqua,MODIS,43,6.3,290.4,5.5
36009,36009,-28.3001,128.5530,303.6,1.0,1.0,2019-09-30,1644,Aqua,MODIS,41,6.3,290.3,5.7


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [16]:
encoder_sat = LabelEncoder()
encoder_int = LabelEncoder()
encoder_conf = LabelEncoder()
encoder_date = LabelEncoder()

df = pd.read_csv('data/df_conf.csv')
df['satellite'] = encoder_sat.fit_transform(df['satellite'])
df['instrument'] = encoder_int.fit_transform(df['instrument'])
df['confidence'] = encoder_conf.fit_transform(df['confidence'])
df[['acq_date']] = df[['acq_date']].astype(str)
df['acq_date'] = encoder_date.fit_transform(df['acq_date'])
